# JAGUAR E-TYPE DATA PULL

In [3]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys
import helpers

Get List of Every Result and Save:

In [105]:
#### Get List of Every Result and Save ####
#### DO NOT DELETE ####

make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/"+ make + model +".html", 'w').close()
ids, urls, t = helpers.getlistings(make, model)
data = ''
listings = []
for i in range(0,len(ids)):
    urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=' + ids[i] + '&sort=td&page='
    urlend = '&results=items'
    pgend = 15
    tempurl = helpers.geturls(urlstart, urlend, pgend, urls)
    listings = tempurl
    listings = listings[len(ids):len(listings)]

    def get_data(url):

        r = requests.get(url)

        soup = BeautifulSoup(r.content, "html.parser")
        return str(soup)


    souplist = joblib.Parallel(n_jobs=-3)(joblib.delayed(get_data)(url) for url in listings)


### WRITE DATA TO FILE ###
    delimiter = "%newline%"
    temp = delimiter.join(souplist)
    data = temp + delimiter + data
with open("./data/"+ make + model +".html", "a") as file:
    file.write(data)


display('done')

'done'

Pull Information Off Each Listing Stored in URLS List:

In [115]:
carlist = []
Car = []
### READ DATA ###

with open("./data/"+ make + model +".html", "r") as file:
    test_data = file.read()

out = test_data.split(delimiter)

for i in range(1,len(listings)):
    soup = BeautifulSoup(out[i], 'html.parser')
    title = str(soup.find('title'))
    if "Parts" not in title and "Tool" not in title and "Memorabilia" not in title and "Tool Kit" not in title and "Removable" not in title and "Gearbox" not in title:
        loc = str(soup.findAll('a'))
        transmission = str(soup.findAll('li'))
        contents = str(soup.findAll('p'))
        essentials = soup.find('div', class_='essentials')

        #Get Car Description
        desc = helpers.getdesc(listings[i])


        #Get Year
        year = desc[0:4]
        
        #Check if Sold and get highest bid
        sold, price = helpers.getsaleprice(title, loc)


        #Get Transmission
        i = 0
        trans = 'Manual'
        if 'Automatic' in transmission:
            trans = 'Automatic'

        #Get Location

        town, state = helpers.getlocation(loc)

        
        #Check for Project
        if 'project' in title or 'Project' in title:
            project = 1
        else:
            project = 0
        
        #Get Lot Number
        try:
            index = title.index('Lot #')
            lotnumb = title[index+5:index+12].partition(')')[0]
        except:
            lotnumb = "N/A"
            
        #Get Month
        month = helpers.getmonth(title)
        
        #Get Year
        if '(' in title:
            index = title.index('(')
            yearsold = title[index-5:index-1]
        else:
            index = title.index('20')
            yearsold = title[index:index+5]
        
        #Get Engine and Number Matching
        numbmatch = 0
        
        if 'Numbers-Matching' in transmission:
            numbmatch = 1

        engine = helpers.getengine(transmission)
        
        #Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        rust, refurbished, restored, scratch, paintbub, metalrepair, hardtop, overdrive, turbo, super = helpers.getindicators(contents)
            
            
        #Get Mileage
        miles, milestmu = helpers.getmileage(transmission)

        #Get Descriptions of options and chassis #

        temp = helpers.getenginedesc(essentials, engine)
        chassis = temp[0]
        specialdesc = temp[1]
        mileagedesc = temp[2]
        enginedesc = temp[3]
        transdesc = temp[4]
        paintdesc = temp[5]
        interiordesc = temp[6]
        carbdesc = temp[7]
        wheeldesc = temp[8]
        brakedesc = temp[9]
        engine = temp[10]

        #Check for bespoke bodies
        if 'Bertone' in title:
            bertone = 1
        else:
            bertone = 0
        if 'Zagato' in title:
            zagato = 1
        else:
            zagato = 0

        Car = [lotnumb, desc, trans, year, miles, milestmu, engine, turbo, super, numbmatch, hardtop, overdrive, scratch, paintbub, metalrepair, rust, refurbished, restored, project, sold, month, yearsold, price, town, state, zagato, bertone, chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc]
        carlist.append(Car)
    else:
        print('Discarded')
    # index = transmission.index('aint')
    # print(transmission[index-20:index+20])

df = pd.DataFrame(carlist, columns=['Lot Number', 'Description', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)','Engine', 'Turbocharged', 'Supercharged', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', 'Rust','Refurbished','Restored', 'Project', 'Sold', 'Month', 'Year', 'Price', 'City', 'State', 'Zagato', 'Bertone', 'Chassis', 'Special Description', 'Mileage Description', 'Engine Description', 'Transmission Description','Paint Description', 'Interior Description','Carburetor Description', 'Wheels Description', 'Brakes Description'])

display(df)



Discarded
Discarded
Discarded


,Lot Number,Description,Transmission,Model Year,Mileage,Mileage (if TMU),Engine,Turbocharged,Supercharged,Numbers Matching,...,Chassis,Special Description,Mileage Description,Engine Description,Transmission Description,Paint Description,Interior Description,Carburetor Description,Wheels Description,Brakes Description
0,"91,661",1970 alfa romeo giulia 1300ti,Manual,1970,TMU,,1.3,0,0,0,...,AR1078836,N/A,"9k Kilometers (~6k Miles) Shown, TMU",1.3L Twin-Cam Inline-Four,Five-Speed Manual Transmission,Blue Paint,Red Vinyl Upholstery,Spare Intake Manifold w/Carburetors,"14"" Steel Wheels",N/A
1,"86,407",1964 alfa romeo guile ti,Manual,1964,TMU,84000,2.0,0,0,0,...,AR448362,N/A,"84k Miles Shown, TMU",2.0-Liter Inline-Four,Five-Speed Manual Transmission,Blue Paint,Gray Vinyl Upholstery,N/A,Five-Spoke 14″ Cromodora Wheels,Four-Wheel Disc Brakes
2,"85,361",1974 alfa romeo giulia 1300 super,Manual,1974,TMU,,1.3,0,0,0,...,AR3340163,N/A,"43k Kilometers (~27k Miles) Shown, TMU","1,290cc Inline-Four",Five-Speed Manual Transmission,Red Paint,Black Vinyl Upholstery,Weber Carburetors,"14"" Wheels",Four-Wheel Disc Brakes
3,"82,571",1964 alfa romeo giulia 1600 ti,Manual,1964,TMU,,1.6,0,0,0,...,AR450626,N/A,"20k Kilometers (12k Miles) Shown, TMU",Twin-Cam 1.6L Inline-Four,Five-Speed Manual Transmission,Gray Paint,Brown Vinyl Upholstery,N/A,"15"" Steel Wheels",Disc Brakes
4,"81,514",1974 alfa romeo giulia nuova super 1300,Manual,1974,,N/A,1.3,0,0,0,...,AR0008467,N/A,"40k Kilometers Shown (~25k Miles), TMU",1.3-Liter Inline-Four,Five-Speed Manual Transmission,Burgundy Paint,Tan Vinyl Upholstery,Dual Carburetors,"14"" Steel Wheels",Four-Wheel Disc Brakes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,"9,083",1985 alfa romeo spider,Manual,1985,39000,N/A,2.0,0,0,0,...,ZARBA5412F1023486,N/A,39k Miles Shown,2.0L Twin-Cam Four,5-Speed Manual,N/A,N/A,N/A,N/A,N/A
921,"8,887",1991 alfa romeo spider,Manual,1991,,N/A,2.0,0,0,0,...,ZARBB32NXM6006703,One North Carolina Owner,"60k Indicated Miles, TMU",2.0L Twin-Cam Inline-Four,5-Speed Manual Transmission,N/A,N/A,N/A,N/A,N/A
922,"8,744",1990 alfa romeo spider,Manual,1990,44467,N/A,2.0,0,0,0,...,ZARBA12G2L6001907,N/A,"44,467 Miles Shown",2.0L Twin-Cam Four,5-Speed Manual,Repainted Last Year,N/A,N/A,N/A,N/A
923,"8,561",1974 alfa romeo spider,Manual,1974,TMU,72000,2.0,0,0,0,...,AR 3044405,N/A,"72k Miles Shown, TMU",2.0L Twin Cam Inline-four,Five-Speed Manual,N/A,N/A,New Weber Carburetors,N/A,N/A


Save Data as Excel File:

In [116]:
path = 'data/' + make + '/'
try:
    os.mkdir(path)
except:
    meme = 0
df.to_excel(path + make + '_' + model + '.xlsx')